In [1]:
import pandas as pd
import json
from openai import OpenAI
from argparse import ArgumentParser
import os.path as osp
import os
import json
import random
import re
import numpy as np
import ast

In [2]:
from movie_pkg.scan_utils import *
from movie_pkg.open_ai_prompts import *

In [6]:
base = "llm_coding" # udpate however necessary
model = "gpt-4.1" # this was the most recent model available via API call when this project was done, but this can likely be updated for future work
# client = OpenAI(
#     api_key=
# )


In [8]:
subjects = get_scan_subs()

### Parsing:
* First, parse the free recall text into individual utterances (typically one agent-action pair)

In [ ]:
base = "" # adjust as necessary

subs = subjects

for subject in subs:
    print(f"running for subject {subject}")
    movie_list = []
    out_dir = f"{base}/llm_coding/scan_parsing/{subject}"
    os.makedirs(out_dir, exist_ok = True)
    for movie in get_subject_movies(subject):
        print(f"     parsing movie {movie}...")
        
        example_data = pd.read_csv(f"{base}/movie_pkg/training/parse_examples/{movie}_example.csv")
        # provide the model example input data
        example_txt = " ".join(example_data['transcript'])
        # provide the model an example of the desired output
        example_out = example_data['transcript']
        # provide the subject's recall as a string
        recall_txt = clean_recall_txt(load_recall_txt(subject, movie))

        # return and save out a list of parsed events and any removed utterances
        result = call_openai_parse(
            client, model, example_txt, example_out, recall_txt
        )

        pd.DataFrame({'utterance': result}).to_csv(
            f"{out_dir}/subj-{subject}_{movie}_parsed.tsv", sep='\t', index=False
        )

        # append output to list for parsing report
        movie_list.append({"movie": movie,
        "recall_segments": result,
        "original_recall_text": recall_txt,
        })
    # generate a parsing report to review identified and removed events
    generate_parsing_report(movie_list, f"{base}/scan_reports/parsing/{subject}_parsing_report.html")

### Event coding:
* Use human generated training data and engineered prompts to align parsed events to reference transcripts

In [ ]:

report_only = False

base = "" # adjust as necessary

for subject in subjects:
    print(f"running for subject {subject}")
    movie_list = []
    out_dir = f"{base}/llm_coding/scan_event_coding_final/{subject}"
    os.makedirs(out_dir, exist_ok = True)
    for movie in get_subject_movies(subject):

        print(f"     running event coding for {movie}...")
        blank_recall_json = get_blank_recall_json(movie)
 
        examples = pull_training_event_coding(movie)

        recall_json = json.loads(recall_tsv_to_json(subject, movie, list_column="utterance"))

        if not report_only:
            # return and save out a list of coded events
            result = call_openai_event_coding_PRESERVE(
                client, model, blank_recall_json, examples, recall_json
            )
            with open(
                f"{out_dir}/subj-{subject}_{movie}_event_coded.json",
                "w",
            ) as f:
                json.dump(result, f, indent=4)
            pass
            tsv_path = f"{out_dir}/subj-{subject}_{movie}_event_coded.tsv"
            json_to_csv(result, tsv_path, subject)
        else:
            tsv_path = f"{out_dir}/subj-{subject}_{movie}_event_coded.tsv"
            results = pd.read_csv(tsv_path, delimiter = '\t')
        
        recall_list = [entry["transcript"] for entry in recall_json] 
        movie_list.append({
                "movie": movie,
                "coded_csv_path": tsv_path,
                "recall_list": recall_list
            })
    generate_coding_report(movie_list, f"{base}/coding_reports/scan/{subject}_coding_report.html")